In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import random


In [2]:
pwd

'/Users/Julien/Documents/VScode/nuclei_quant/notebooks'

In [ ]:
# file path of all csv:
files = '../img_test_pipeline'

In [4]:
# Dictionary of the clones and their corresponding genotypes: CHANGE IT ACCORDINGLY TO YOURS!!
genotypes = {
    'Pa':'WT/WT',
    'B6WT': 'WT/WT',
    'H6':'KO/KO',
    'C6':'KO/KO'
}

In [5]:
# Count the total number of CSV files
total_files = sum(1 for filename in os.listdir(files) if filename.endswith('.csv') and not filename.startswith("._"))
print(f'You have {total_files} files in your folder')

You have 12 files in your folder


In [6]:
# Extract the data from one random file to see the correpsonding matching
csv_files = [f for f in os.listdir(files) if f.endswith('.csv') and not f.startswith("._")]
rfilename = random.choice(csv_files)


# Data extraction from the filename:
parts = rfilename.split('_')
clone, dif, day, immuno, obj, imaging, og, s = parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7]
print(f"clone: {clone}, dif: {dif}, day: {day}, immuno: {immuno}, obj: {obj}, imaging: {imaging}, og: {og}, s: {s}")

clone: Pa, dif: dQ, day: J70, immuno: SNd2T, obj: 20Xa, imaging: g1dt08, og: OG2, s: s6


In [7]:
# Create an empty list to store temporarly the information before the df concatenation:
dfs = []
with tqdm(total=total_files, ncols=80) as pbar:
    for filename in os.listdir(files):
        if filename.endswith('.csv') and not filename.startswith("._"):
            
            # Read the csv files
            filepath = os.path.join(files, filename)
            df = pd.read_csv(filepath)

            #Data extraction from the filename:
            parts = filename.split('_')
            clone, dif, day, immuno, obj, imaging, og, s = parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7]
            # Add additional information to the df 
            df['Image_name'] = filename
            df['Clone'] = clone
            df['Diff'] = dif
            df['Day'] = day
            df['Immuno'] = immuno
            df['Objective'] = obj
            df['Imaging'] = imaging
            #df['Settings'] = settings
            df['Organoid'] = og
            df['Slice'] = s
            df['Genotype'] = df['Clone'].map(genotypes)

            # Append the dfs:
            dfs.append(df)

            # Update the progress bar
            pbar.update(1)

# Combine all the DataFrames:
combined_df = pd.concat(dfs, ignore_index=True)

# Define the order of columns: metadata columns followed by remaining columns
metadata_columns = ['Image_name', 'Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Objective', 'Imaging', 'Organoid', 'Slice']
existing_columns = [col for col in combined_df.columns if col not in metadata_columns]

# Column reordering
columns_order = metadata_columns + existing_columns
combined_df = combined_df.reindex(columns=columns_order)

100%|███████████████████████████████████████████| 12/12 [00:04<00:00,  2.66it/s]


In [8]:
# Verify your df before saving
combined_df.head()

,Image_name,Clone,Genotype,Diff,Day,Immuno,Objective,Imaging,Organoid,Slice,...,PrincipalAxes_14,PrincipalAxes_15,Object Area,Convexity,Bounding Box Minimum_0,Bounding Box Minimum_1,Bounding Box Maximum_0,Bounding Box Maximum_1,Center of the object_0,Center of the object_1
0,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2_table.csv,Pa,WT/WT,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s2,...,0.399915,0.891539,33.0,0.916667,1557.0,41.0,1563.0,47.0,1559.72730,43.545456
1,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2_table.csv,Pa,WT/WT,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s2,...,-0.100134,0.993293,33.0,0.916667,2175.0,52.0,2181.0,58.0,2177.57570,54.424244
2,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2_table.csv,Pa,WT/WT,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s2,...,-0.154524,-0.987384,32.0,0.888889,145.0,87.0,151.0,93.0,147.18750,89.468750
3,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2_table.csv,Pa,WT/WT,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s2,...,-0.031199,-0.997387,47.0,0.839286,140.0,93.0,148.0,100.0,143.63829,95.702126
4,Pa_dQ_J70_SNd2T_20Xa_g1dt08_OG1_s2_table.csv,Pa,WT/WT,dQ,J70,SNd2T,20Xa,g1dt08,OG1,s2,...,0.192376,-0.981062,28.0,0.777778,309.0,122.0,315.0,128.0,311.28570,124.821430


In [ ]:
# To use only when you already have the Raw classification csv file
#combined_df = pd.read_csv('//l2export/iss02.hassan/analyses/julien.pigeon/analyses/Raw_classifications/Raw_classification_dM_J70_SNT.csv')

In [10]:
# Group by Image_name, Predicted Class, clone, genotype, organoid, and slice, then count the occurrences of each combination
cell_counts = combined_df.groupby(['Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Organoid', 'Slice', 'Predicted Class',]).size().reset_index(name='Cell Count')

# Pivot the table to have predicted classes as columns
pivot_table = cell_counts.pivot_table(index=['Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Organoid', 'Slice'], columns='Predicted Class', values='Cell Count', fill_value=0)

# Reset index to make Image_name a regular column
pivot_table.reset_index(inplace=True)

# print the pivot table
pivot_table

Predicted Class,Clone,Genotype,Diff,Day,Immuno,Organoid,Slice,Dead,N,S,T
0,Pa,WT/WT,dQ,J70,SNd2T,OG1,s1,23897.0,3342.0,10693.0,1090.0
1,Pa,WT/WT,dQ,J70,SNd2T,OG1,s2,42433.0,5153.0,11511.0,1939.0
2,Pa,WT/WT,dQ,J70,SNd2T,OG1,s3,40982.0,8159.0,12362.0,2009.0
3,Pa,WT/WT,dQ,J70,SNd2T,OG1,s4,47437.0,6328.0,11072.0,2727.0
4,Pa,WT/WT,dQ,J70,SNd2T,OG2,s1,10788.0,6753.0,6956.0,2047.0
5,Pa,WT/WT,dQ,J70,SNd2T,OG2,s2,36603.0,7919.0,10749.0,1788.0
6,Pa,WT/WT,dQ,J70,SNd2T,OG2,s3,37756.0,9897.0,9381.0,3010.0
7,Pa,WT/WT,dQ,J70,SNd2T,OG2,s4,36450.0,9264.0,10476.0,3438.0
8,Pa,WT/WT,dQ,J70,SNd2T,OG2,s6,23994.0,9202.0,7980.0,3794.0
9,Pa,WT/WT,dQ,J70,SNd2T,OG3,s2,21187.0,5306.0,11044.0,2800.0


In [11]:
import openpyxl
#pivot_table.to_excel(f'//iss/hassan/analyses/julien.pigeon/analyses/Tables/haug2/{dif}_{day}_{immuno}_slices.xlsx', index=False)
pivot_table.to_excel(f'../data/{dif}_{day}_{immuno}_slices.xlsx', index=False)

In [12]:
# Group by Image_name, Organoid, Predicted Class, then count the occurrences of each combination
cell_counts = combined_df.groupby(['Clone', 'Genotype', 'Diff','Day', 'Immuno', 'Organoid', 'Predicted Class']).size().reset_index(name='Cell Count')

# Pivot the table to have predicted classes as columns
pivot_table = cell_counts.pivot_table(index=['Clone', 'Genotype', 'Diff','Day', 'Immuno', 'Organoid'], columns='Predicted Class', values='Cell Count', fill_value=0)

# Reset index to make Image_name and Organoid regular columns
pivot_table.reset_index(inplace=True)

# print the pivot table
pivot_table

Predicted Class,Clone,Genotype,Diff,Day,Immuno,Organoid,Dead,N,S,T
0,Pa,WT/WT,dQ,J70,SNd2T,OG1,154749.0,22982.0,45638.0,7765.0
1,Pa,WT/WT,dQ,J70,SNd2T,OG2,145591.0,43035.0,45542.0,14077.0
2,Pa,WT/WT,dQ,J70,SNd2T,OG3,79790.0,19671.0,30951.0,9512.0


In [13]:
pivot_table.to_excel(f'../data/{dif}_{day}_{immuno}_OG.xlsx', index=False)

Curate the data to have at least 4 slices per organoids

In [14]:
cell_counts = combined_df.groupby(['Clone', 'Genotype', 'Diff','Day', 'Immuno', 'Organoid', 'Predicted Class']).size().reset_index(name='Cell Count')

# Step 1: Count the number of unique slices for each organoid
slice_counts = combined_df.groupby(['Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Organoid'])['Slice'].nunique().reset_index(name='Slice Count')

# Step 2: Filter out organoids with fewer than 4 slices
valid_organoids = slice_counts[slice_counts['Slice Count'] >= 4]

# Step 3: Merge the valid organoids back into the main data
filtered_data = cell_counts.merge(valid_organoids, on=['Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Organoid'], how='inner')


# Step 5: Create the pivot table grouped by organoid
pivot_table = filtered_data.pivot_table(
    index=['Clone', 'Genotype', 'Diff', 'Day', 'Immuno', 'Organoid'],
    columns='Predicted Class',
    values='Cell Count',
    fill_value=0
)

# Reset index to make it a regular column
pivot_table.reset_index(inplace=True)

# Print the pivot table
pivot_table


Predicted Class,Clone,Genotype,Diff,Day,Immuno,Organoid,Dead,N,S,T
0,Pa,WT/WT,dQ,J70,SNd2T,OG1,154749.0,22982.0,45638.0,7765.0
1,Pa,WT/WT,dQ,J70,SNd2T,OG2,145591.0,43035.0,45542.0,14077.0


In [15]:
import openpyxl
pivot_table.to_excel(f'../data/{dif}_{day}_{immuno}_OG_curated.xlsx', index=False)


OG 3 was excluded from the analysis because organoids with fewer than 4 slices were discarded to ensure complete depth coverage.

In [ ]:
# Save your the raw data after running the object classifier. This might take some time to save. 
combined_df.to_csv(f'../data/Raw_classification_{dif}_{day}_{immuno}.csv', index=False)
